In [10]:
# imports

import os
import data_utils

# Detectron related imports
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
import matplotlib.pyplot as plt

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.structures import BoxMode


In [11]:
# yolo data format (all images in one folder, all labels in one folder and no train val test split)

yolo_images = './All_Datasets/filtered_ls_data/images'
yolo_labels = './All_Datasets/filtered_ls_data/labels'

datasets = 'All_Datasets'



In [12]:
# path to an image
image_path = 'All_Datasets/filtered_ls_data/images/LT05_L1TP_130038_19860807_20170221_01_T1_patch_1920_5120.jpeg'
label_path = 'All_Datasets/filtered_ls_data/labels/LT05_L1TP_130038_19860807_20170221_01_T1_patch_1920_5120.txt'

# get the dimensions of the image
print(data_utils.get_image_dimensions(image_path))

(640, 640, 3)


In [13]:

# get the list of images and labels
images = os.listdir(yolo_images)
labels = os.listdir(yolo_labels)
# get the list of image ids
image_ids = [image.split('.')[0] for image in images]
# get the list of label ids
label_ids = [label.split('.')[0] for label in labels]

print(image_ids)
print(label_ids)

# check if all the images have labels
if set(image_ids) == set(label_ids):
    print('All images have labels')
else:
    print('Some images do not have labels')


    
# Obtain one label to check the format
with open(os.path.join(label_dir, filename.split('.')[0] + '.txt')) as f:
    labels = [line.rstrip('\n') for line in open(os.path.join(label_dir, filename.split('.')[0] + '.txt'))]

#split label string into list of floats
label_float= list(map(float, labels[0].split()))

print(label_float[1:])

['LT05_L1TP_130039_19880202_20170221_01_T1_patch_4480_640', 'LT05_L1TP_129038_19860731_20170221_01_T1_patch_3200_640', 'LT05_L1TP_130038_19891205_20170214_01_T2_patch_3840_4480', 'LT05_L1TP_130038_19891205_20170214_01_T2_patch_3840_3200', 'LT05_L1TP_130039_19880202_20170221_01_T1_patch_4480_1280', 'LT05_L1TP_139041_20101230_20161011_01_T1_patch_3840_3200', 'LT05_L1TP_130038_20070918_20161111_01_T1_patch_5120_3200', 'LT05_L1TP_130038_20070918_20161111_01_T1_patch_1920_3840', 'LT05_L1TP_130038_20070918_20161111_01_T1_patch_5120_4480', 'LT05_L1TP_130038_19860807_20170221_01_T1_patch_4480_1920', 'LT05_L1TP_130038_20070918_20161111_01_T1_patch_2560_5120', 'LT05_L1TP_130038_20070918_20161111_01_T1_patch_3200_5120', 'LT05_L1TP_130038_20070918_20161111_01_T1_patch_3200_3200', 'LT05_L1TP_130038_20070918_20161111_01_T1_patch_3200_4480', 'LT05_L1TP_129038_19860731_20170221_01_T1_patch_1280_3200', 'LT05_L1TP_129038_19860731_20170221_01_T1_patch_1920_1920', 'LT05_L1TP_130038_20070918_20161111_01_T1

NameError: name 'filename' is not defined

In [14]:
# get path to the image
img_dir = 'All_Datasets/filtered_ls_data/images'
label_dir = 'All_Datasets/filtered_ls_data/labels'

def convert_yolo_to_detectron2_coco(img_dir, label_dir):

    # get the list of images
    images = os.listdir(img_dir)
    # get the list of labels
    labels = os.listdir(label_dir)

    # Final dataset dicts - what should be returned
    dataset_dicts = []

    # image iterator:
    for i in range(len(images)):

        # deal with one image at a time:        
        record = {}
        objs = [] # Corresponding to the annotations/instances of objects in the image; list[float] is one simple polygon
        filename = images[i]
        height, width, _ = data_utils.get_image_dimensions(os.path.join(img_dir, filename))

        # Load labels
        with open(os.path.join(label_dir, filename.split('.')[0] + '.txt')) as f:
            labels = [line.rstrip('\n') for line in open(os.path.join(label_dir, filename.split('.')[0] + '.txt'))]

        # iterate through the labels
        for label in labels:

            # get the class id:
            class_id = int(label.split(' ')[0])

            # Converting string labels list to floats
            label_float= list(map(float, label.split()))
            flat_coordinates = label_float[1:]

            # Obtaining the polygon coordinates:
            poly = [] #list[float] is one simple polygon in the format of [x1, y1, ..., xn, yn] (n≥3). The Xs and Ys are absolute coordinates in unit of pixels.

            # Scaling up the coordinates to x_scale and y_scale to add to the polygon
            for i in range(len(flat_coordinates)):

                # x values
                if i % 2 == 0:
                    poly.append(flat_coordinates[i] * width) # Append to the polygon

                # y values
                else:
                    poly.append(flat_coordinates[i] * height) # Append to the polygon

            obj = {
                "bbox": data_utils.get_bounding_box_from_yolo_label(flat_coordinates),
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": class_id,
                }
                
            objs.append(obj)

        # Adding to the record:
        record["file_name"] = filename
        record["image_id"] = i
        record["height"] = height
        record["width"] = width
        record["annotations"] = objs

        dataset_dicts.append(record)

    return dataset_dicts






# get the list of image annotations
# what is the image annotation?
# image annotation is a list of dictionaries, each dictionary contains the following keys:
# 'id', 'file_name', 'width', 'height'
# the value of 'id' is the image id
# the value of 'file_name' is the image name
# the value of 'width' is the image width
# the value of 'height' is the image height

# get the list of image annotations (given the image path, label path and datasets from scratch)


In [15]:
for d in ["train"]:
    DatasetCatalog.register("landslide_" + d, lambda d=d: convert_yolo_to_detectron2_coco(img_dir, label_dir))
    MetadataCatalog.get("landslide_" + d).set(thing_classes=["landslide"])
landslide_metadata = MetadataCatalog.get("landslide_train")

print(landslide_metadata)

Metadata(name='landslide_train', thing_classes=['landslide'])


In [16]:
print(balloon_metadata)
print(landslide_metadata)

NameError: name 'balloon_metadata' is not defined

In [19]:
# Checking with detectron2

# get path to the image
img_dir = 'All_Datasets/filtered_ls_data/images'
label_dir = 'All_Datasets/filtered_ls_data/labels'

dataset_dicts = convert_yolo_to_detectron2_coco(img_dir, label_dir)
print(len(dataset_dicts))
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(os.path.join(img_dir, d["file_name"]))
    visualizer = Visualizer(img[:, :, ::-1], metadata=landslide_metadata)
    out = visualizer.draw_dataset_dict(d)
    # show image
    cv2.imshow("Sample Image", out.get_image()[:, :, ::-1])  # Added a window name here
    cv2.waitKey(0)  # Wait for a key press to close the window

219


: 

In [8]:
images = os.listdir(img_dir)
print(len(images))

219
